In [13]:
import os

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.v1.extensions.google_cloud_big_query import BigQueryExampleGen
from tfx.v1.components import StatisticsGen, SchemaGen, Transform, Trainer

# Project / BigQuery config
PROJECT_ID = "uci-bank-marketing-ml-lab"
GCP_REGION = "us-central1"

BQ_PROJECT = PROJECT_ID
BQ_DATASET = "bank_data"
BQ_TABLE = "uci_bank_marketing"

# Local pipeline root (same as previous notebooks)
PIPELINE_NAME = "bq_classifier_validation"
LOCAL_PIPELINE_ROOT = os.path.join(os.getcwd(), "tfx_local", PIPELINE_NAME)
os.makedirs(LOCAL_PIPELINE_ROOT, exist_ok=True)

# Env vars (same as earlier)
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["GCLOUD_PROJECT"] = PROJECT_ID

# Interactive context
context = InteractiveContext(pipeline_root=LOCAL_PIPELINE_ROOT)
print("Using pipeline_root:", context.pipeline_root)

# Beam / Dataflow temp location (same bucket as before)
TMP_LOCATION = "gs://uci-bank-ml-bucket/tfx/tmp"

beam_pipeline_args = [
    f"--project={PROJECT_ID}",
    f"--region={GCP_REGION}",
    f"--temp_location={TMP_LOCATION}",
]


Using pipeline_root: /home/jupyter/tfx_local/bq_classifier_validation


#### BigQueryExampleGen 

In [14]:
from tfx.v1.extensions.google_cloud_big_query import BigQueryExampleGen

# Simple query that matches previous ingestion
query = f"""
SELECT *
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
"""

example_gen = BigQueryExampleGen(query=query)

# This will create / reuse Examples artifacts in tfx_local/bq_classifier_validation/BigQueryExampleGen
context.run(example_gen, beam_pipeline_args=beam_pipeline_args)


ExecutionResult(
    component_id: BigQueryExampleGen
    execution_id: 43
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=BigQueryExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

#### StatisticsGen

In [15]:
from tfx.v1.components import StatisticsGen

statistics_gen = StatisticsGen(examples=example_gen.outputs["examples"])
context.run(statistics_gen, beam_pipeline_args=beam_pipeline_args)


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 44
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

#### SchemaGen

In [16]:
from tfx.v1.components import SchemaGen

schema_gen = SchemaGen(
    statistics=statistics_gen.outputs["statistics"],
    infer_feature_shape=True,
)
context.run(schema_gen, beam_pipeline_args=beam_pipeline_args)


ExecutionResult(
    component_id: SchemaGen
    execution_id: 45
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

#### Transform

In [17]:
from tfx.v1.components import Transform
import os

transform = Transform(
    examples=example_gen.outputs["examples"],
    schema=schema_gen.outputs["schema"],
    module_file=os.path.join(os.getcwd(), "transform_module.py"),
)

transform_result = context.run(transform, beam_pipeline_args=beam_pipeline_args)

print("Transform outputs:")
print(transform.outputs)


running bdist_wheel
running build
running build_py
creating build/lib
copying trainer_module.py -> build/lib
copying transform_module.py -> build/lib
installing to /var/tmp/tmp6p50d_qm
running install
running install_lib
copying build/lib/transform_module.py -> /var/tmp/tmp6p50d_qm/.
copying build/lib/trainer_module.py -> /var/tmp/tmp6p50d_qm/.
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /var/tmp/tmp6p50d_qm/./tfx_user_code_Transform-0.0+95f8d70eb9c6528bbbc86b56fdfe26bce5a8131d9acaa26df9948ca666c65

/home/jupyter/tfx-env/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:90: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        This deprecation is overdue, please update your project and remove deprecated
        calls to avoid build errors in the future.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


Processing ./tfx_local/bq_classifier_validation/_wheels/tfx_user_code_transform-0.0+95f8d70eb9c6528bbbc86b56fdfe26bce5a8131d9acaa26df9948ca666c65545-py3-none-any.whl
Processing ./tfx_local/bq_classifier_validation/_wheels/tfx_user_code_transform-0.0+95f8d70eb9c6528bbbc86b56fdfe26bce5a8131d9acaa26df9948ca666c65545-py3-none-any.whl
Processing ./tfx_local/bq_classifier_validation/_wheels/tfx_user_code_transform-0.0+95f8d70eb9c6528bbbc86b56fdfe26bce5a8131d9acaa26df9948ca666c65545-py3-none-any.whl


Transform outputs:
{'transform_graph': OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False), 'transformed_examples': OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False), 'updated_analyzer_cache': OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False), 'pre_transform_schema': OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False), 'pre_transform_stats': OutputChannel(artifact_type=ExampleStatistics, pr

In [23]:
%%writefile trainer_module.py
import tensorflow as tf
import tensorflow_transform as tft

LABEL_KEY = "y"


def _get_feature_keys_from_schema(schema):
    """Collect all transformed feature names except the label."""
    feature_keys = []
    for feature in schema.feature:
        if feature.name == LABEL_KEY:
            continue
        feature_keys.append(feature.name)
    return feature_keys


def _make_file_list(file_input):
    """Normalize fn_args.train_files / eval_files to a flat list of files."""
    if isinstance(file_input, str):
        patterns = [file_input]
    else:
        patterns = list(file_input)

    files = []
    for p in patterns:
        files.extend(tf.io.gfile.glob(p))
    return files


def _input_fn(file_input, tf_transform_output, batch_size=128):
    """Builds an input_fn reading transformed TFRecords (supports GZIP)."""
    transformed_feature_spec = tf_transform_output.transformed_feature_spec()

    files = _make_file_list(file_input)
    if not files:
        raise ValueError(f"No TFRecord files found for pattern(s): {file_input}")

    # Detect compression (TFX often writes gzip-compressed TFRecords)
    compression_type = None
    if any(f.endswith(".gz") for f in files):
        compression_type = "GZIP"

    dataset = tf.data.TFRecordDataset(
        files,
        compression_type=compression_type
    )

    def _parse_fn(serialized_example):
        return tf.io.parse_single_example(serialized_example, transformed_feature_spec)

    dataset = dataset.map(_parse_fn, num_parallel_calls=tf.data.AUTOTUNE)

    def _split_features_label(example):
        label = example.pop(LABEL_KEY)
        label = tf.cast(label, tf.float32)
        if len(label.shape) == 0:
            label = tf.reshape(label, [1])
        return example, label

    dataset = dataset.map(_split_features_label,
                          num_parallel_calls=tf.data.AUTOTUNE)

    # Repeat so we always have enough batches for steps_per_epoch
    dataset = dataset.shuffle(10000).repeat().batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset


def _build_keras_model(feature_keys):
    """Simple DNN over concatenated transformed features."""
    inputs = {
        key: tf.keras.Input(shape=(1,), name=key, dtype=tf.float32)
        for key in feature_keys
    }

    x = tf.keras.layers.Concatenate()(list(inputs.values()))
    x = tf.keras.layers.Dense(64, activation="relu")(x)
    x = tf.keras.layers.Dense(32, activation="relu")(x)
    outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="binary_crossentropy",
        metrics=[tf.keras.metrics.AUC(name="auc"), "accuracy"],
    )
    return model


class _ServingModule(tf.Module):
    """Wrapper module that holds both the model and the Transform layer."""

    def __init__(self, model, transform_layer, raw_feature_spec):
        super().__init__()
        # These are tracked resources/variables
        self.model = model
        self.transform_layer = transform_layer
        self.raw_feature_spec = raw_feature_spec

    @tf.function(
        input_signature=[
            tf.TensorSpec(shape=[None], dtype=tf.string, name="examples")
        ]
    )
    def serve_tf_examples_fn(self, serialized_tf_examples):
        """Serving fn: tf.Example -> transformed features -> model -> predictions."""
        # Parse raw features from tf.Example
        raw_features = tf.io.parse_example(serialized_tf_examples,
                                           self.raw_feature_spec)

        # Apply Transform graph
        transformed_features = self.transform_layer(raw_features)

        # Call the Keras model (expects transformed features dict)
        outputs = self.model(transformed_features)

        # TFMA & Pusher expect a dict of outputs
        return {"output_0": outputs}


def run_fn(fn_args):
    """TFX Trainer entrypoint."""
    # 1) Load transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    # 2) Figure out which transformed features to feed the model
    transformed_schema = tf_transform_output.transformed_metadata.schema
    feature_keys = _get_feature_keys_from_schema(transformed_schema)

    # 3) Build model
    model = _build_keras_model(feature_keys)

    # 4) Build datasets
    train_ds = _input_fn(fn_args.train_files, tf_transform_output, batch_size=128)
    eval_ds = _input_fn(fn_args.eval_files, tf_transform_output, batch_size=128)

    # 5) Train
    model.fit(
        train_ds,
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_ds,
        validation_steps=fn_args.eval_steps,
    )

    # 6) Prepare objects needed for serving
    transform_layer = tf_transform_output.transform_features_layer()
    raw_feature_spec = tf_transform_output.raw_feature_spec()

    # Remove the label from serving inputs (we don't send labels at serving time)
    if LABEL_KEY in raw_feature_spec:
        raw_feature_spec.pop(LABEL_KEY)

    # 7) Wrap into a tf.Module that tracks both model and transform layer
    serving_module = _ServingModule(
        model=model,
        transform_layer=transform_layer,
        raw_feature_spec=raw_feature_spec,
    )

    # 8) Export SavedModel with a tf.Example-based serving_default signature
    tf.saved_model.save(
        obj=serving_module,
        export_dir=fn_args.serving_model_dir,
        signatures={
            "serving_default": serving_module.serve_tf_examples_fn
        },
    )


Overwriting trainer_module.py


#### Trainer 

In [24]:
# Rebuild wheel + rerun Trainer
trainer = Trainer(
    module_file=os.path.join(os.getcwd(), "trainer_module.py"),
    examples=transform.outputs["transformed_examples"],
    transform_graph=transform.outputs["transform_graph"],
    train_args={'num_steps': 2000},
    eval_args={'num_steps': 500},
)

trainer_result = context.run(trainer, beam_pipeline_args=beam_pipeline_args)

print("Trainer outputs:", trainer.outputs)


running bdist_wheel
running build
running build_py
creating build/lib
copying trainer_module.py -> build/lib
copying transform_module.py -> build/lib
installing to /var/tmp/tmpjuwsbl6j
running install
running install_lib
copying build/lib/transform_module.py -> /var/tmp/tmpjuwsbl6j/.
copying build/lib/trainer_module.py -> /var/tmp/tmpjuwsbl6j/.
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /var/tmp/tmpjuwsbl6j/./tfx_user_code_Trainer-0.0+0f8c7c9fcd437087ec2d8b8b16b012c2be2b30424f52706cc97d9564fc8751c8-py3.10.egg-inf

/home/jupyter/tfx-env/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:90: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        This deprecation is overdue, please update your project and remove deprecated
        calls to avoid build errors in the future.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


Processing ./tfx_local/bq_classifier_validation/_wheels/tfx_user_code_trainer-0.0+0f8c7c9fcd437087ec2d8b8b16b012c2be2b30424f52706cc97d9564fc8751c8-py3-none-any.whl
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 31s 15ms/step - accuracy: 0.9122 - auc: 0.9339 - loss: 0.1929 - val_accuracy: 0.9132 - val_auc: 0.9376 - val_loss: 0.1853
Trainer outputs: {'model': OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False), 'model_run': OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)}


In [25]:
trainer.outputs["model"]


OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)

In [26]:
import tensorflow as tf
import os

saved_model_dir = "/home/jupyter/tfx_local/bq_classifier_validation/Trainer/model/30/Format-Serving"
loaded = tf.saved_model.load(saved_model_dir)

serve_fn = loaded.signatures["serve"]
print(serve_fn) 


ConcreteFunction Input Parameters:
  age_z (KEYWORD_ONLY): TensorSpec(shape=(None, 1), dtype=tf.float32, name='age_z')
  campaign_z (KEYWORD_ONLY): TensorSpec(shape=(None, 1), dtype=tf.float32, name='campaign_z')
  cons_conf_idx_z (KEYWORD_ONLY): TensorSpec(shape=(None, 1), dtype=tf.float32, name='cons_conf_idx_z')
  cons_price_idx_z (KEYWORD_ONLY): TensorSpec(shape=(None, 1), dtype=tf.float32, name='cons_price_idx_z')
  contact_id (KEYWORD_ONLY): TensorSpec(shape=(None, 1), dtype=tf.float32, name='contact_id')
  day_of_week_id (KEYWORD_ONLY): TensorSpec(shape=(None, 1), dtype=tf.float32, name='day_of_week_id')
  default_id (KEYWORD_ONLY): TensorSpec(shape=(None, 1), dtype=tf.float32, name='default_id')
  duration_z (KEYWORD_ONLY): TensorSpec(shape=(None, 1), dtype=tf.float32, name='duration_z')
  education_id (KEYWORD_ONLY): TensorSpec(shape=(None, 1), dtype=tf.float32, name='education_id')
  emp_var_rate_z (KEYWORD_ONLY): TensorSpec(shape=(None, 1), dtype=tf.float32, name='emp_var_ra

In [27]:
import numpy as np

sample = {
    "age_z": tf.constant([[0.0]], dtype=tf.float32),
    "campaign_z": tf.constant([[0.0]], dtype=tf.float32),
    "cons_conf_idx_z": tf.constant([[0.0]], dtype=tf.float32),
    "cons_price_idx_z": tf.constant([[0.0]], dtype=tf.float32),
    "contact_id": tf.constant([[1.0]], dtype=tf.float32),
    "day_of_week_id": tf.constant([[1.0]], dtype=tf.float32),
    "default_id": tf.constant([[0.0]], dtype=tf.float32),
    "duration_z": tf.constant([[0.0]], dtype=tf.float32),
    "education_id": tf.constant([[2.0]], dtype=tf.float32),
    "emp_var_rate_z": tf.constant([[0.0]], dtype=tf.float32),
    "euribor3m_z": tf.constant([[0.0]], dtype=tf.float32),
    "housing_id": tf.constant([[1.0]], dtype=tf.float32),
    "job_id": tf.constant([[3.0]], dtype=tf.float32),
    "loan_id": tf.constant([[0.0]], dtype=tf.float32),
    "marital_id": tf.constant([[1.0]], dtype=tf.float32),
    "month_id": tf.constant([[5.0]], dtype=tf.float32),
    "nr_employed_z": tf.constant([[0.0]], dtype=tf.float32),
    "pdays_z": tf.constant([[0.0]], dtype=tf.float32),
    "poutcome_id": tf.constant([[0.0]], dtype=tf.float32),
    "previous_z": tf.constant([[0.0]], dtype=tf.float32),
}

pred = serve_fn(**sample)
print("Prediction:", pred)


Prediction: {'output_0': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.06804091]], dtype=float32)>}
